## 示例选择器使用
- 根据长度动态选择
- 根据语义相似度动态选择（最大余弦）
- 使用最大边际相关性进行选择（MMR）

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

### 根据长度动态选择
根据用户的输入、提示词总长度结合模型窗口大小来动态计算可以容纳的示例个数
原理：
    根据示例的长度来选择。你可以选择那些长度最短的示例（如果提示词长度有限），或者最长的示例（如果需要更全面的上下文）。
场景：
    当你需要确保整个提示词的总长度不超过 LLM 的上下文窗口限制时，这种方法非常有用。

In [2]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 假设已经有的提示词示例组，示例是大模型的输入输出
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tired", "output": "sleepy"},
    {"input": "frustrated", "output": "bored"},
    {"input": "sick", "output": "healthy"},
    {"input": "hungry", "output": "full"},
    {"input": "lonely", "output": "connected"},
    {"input": "angry", "output": "calm"},
    {"input": "stressed", "output": "calm"},
    {"input": "excited", "output": "calm"},
    {"input": "scared", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# 构建提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input}\n反义词: {output}",
)

# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    examples=examples,  # 传入提示词示例组
    example_prompt=example_prompt,  # 传入提示词模板
    max_length=25,  # 设置格式化后的提示词最大长度
    # 设置获取长度的方法，不设置默认使用get_text_length，如果默认分词器计算不满足，可以自己扩展
    # get_text_length=lambda x: len(x),  # 设置获取文本长度的函数
)


# 使用示例提示词模板来实现动态示例选择器的调用
dynmic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 使用示例选择器
    example_prompt=example_prompt,  # 使用示例提示词模板
    prefix="请给出输入的反义词",  # 设置前缀提示词
    suffix="原词: {input}\n反义词:",  # 设置后缀提示词
    input_variables=["input"],  # 设置输入变量
)

# 打印调用示例选择器之后的模板
print(dynmic_prompt)

# 格式化提示词
print(dynmic_prompt.format(input="sad"))

input_variables=['input'] input_types={} partial_variables={} example_selector=LengthBasedExampleSelector(examples=[{'input': 'happy', 'output': 'sad'}, {'input': 'tired', 'output': 'sleepy'}, {'input': 'frustrated', 'output': 'bored'}, {'input': 'sick', 'output': 'healthy'}, {'input': 'hungry', 'output': 'full'}, {'input': 'lonely', 'output': 'connected'}, {'input': 'angry', 'output': 'calm'}, {'input': 'stressed', 'output': 'calm'}, {'input': 'excited', 'output': 'calm'}, {'input': 'scared', 'output': 'calm'}, {'input': '高兴', 'output': '悲伤'}], example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='原词: {input}\n反义词: {output}'), get_text_length=<function _get_length_based at 0x750335c0ec00>, max_length=25, example_text_lengths=[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]) example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='原词: {input}\n反义词: {output}') suffix='原词: {input}\n反义词

### 根据语义相似度动态选择
筛选示例组中与输入的语义相似度最高的示例
本质：将问题与示例嵌入向量空间后进行搜索比对
依赖：向量数据库

原理：
    这是最常用、最强大的方法。它将用户输入和示例库中的所有示例都转换为向量嵌入（Vector Embeddings），然后计算向量之间的相似度。它会选择与用户输入最相似的几个示例。
场景：
    当你希望 LLM 能够根据最相关的上下文进行推理时，这种方法最有效。例如，在问答任务中，它能找到与当前问题最相似的过往问答示例。

In [3]:
# 选择chromadb向量数据库进行向量化和向量比较
! pip install chromadb==0.4.15

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
# 使用最大余弦相似度来检索相关示例，以便示例尽量贴近于输入的内容
# 在向量空间中查找最近的

from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import TongyiEmbeddings

model = os.environ.get("TONGYI_MODEL")
base_url = os.environ.get("TONGYI_API_BASE")
api_key = os.environ.get("TONGYI_API_KEY")

embeddings = OpenAIEmbeddings(
    model=model,
    openai_api_base=base_url,
    openai_api_key=api_key,
)


# 构建提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input}\n反义词: {output}",
)

# 一组包含各种性质的反义词示例
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tired", "output": "sleepy"},
    {"input": "frustrated", "output": "bored"},
    {"input": "sick", "output": "healthy"},
    {"input": "hungry", "output": "full"},
    {"input": "lonely", "output": "connected"},
    {"input": "angry", "output": "calm"},
    {"input": "stressed", "output": "calm"},
    {"input": "excited", "output": "calm"},
    {"input": "scared", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,  # 设置示例组
    embeddings=embeddings,  # 向量化使用的嵌入模型
    vectorstore_cls=Chroma,  # 设置向量库
    k=1,  # 获取相似度最高的数量
)

# 使用示例提示词模板来实现动态示例选择器的调用
dynmic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 使用示例选择器
    example_prompt=example_prompt,  # 使用示例提示词模板
    prefix="请给出输入的反义词",  # 设置前缀提示词
    suffix="原词: {input}\n反义词:",  # 设置后缀提示词
    input_variables=["input"],  # 设置输入变量
)

# 使用
print(dynmic_prompt.format(input="sad"))

ValidationError: 1 validation error for OpenAIEmbeddings
model
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

### 按最大边际相关性选择（Maximal Marginal Relevance, MMR）
原理：
    MMR 是一种更高级的相似性选择。它不仅考虑示例与用户输入的相似性，还考虑被选中的示例之间的差异性。MMR 试图在相关性和多样性之间找到平衡，以避免选择多个内容高度相似的示例。
    MMR 是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡。MMR会首先找出与输入最相似（即余弦相似度最大）的样本。然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本惩罚。MMR既能确保选出来的样本于输入高度相关，又能保证选出的样本之间有足够的多样性。关注如何在相关性和多样性之间找到平衡。
场景：
    当你需要提供一组多样化但又都与问题相关的示例时，MMR 效果最好。这可以防止模型过拟合于某个特定的示例类型。
依赖：向量数据库

In [4]:
# 这里采用FAISS向量数据库进行向量化和向量比较
! pip install faiss-cpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 813.7 kB/s  0:00:370:00:0100:02


In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# 示例库
examples = [
    {
        "input": "我在哪里可以找到我的行李？",
        "output": "请提供您的航班号，以便我查询您的行李。",
    },
    {"input": "我的行李丢了怎么办？", "output": "请立即联系机场失物招领处。"},
    {
        "input": "我想报告一件丢失的物品。",
        "output": "请说明丢失的物品，以及是在哪个航站楼或登机口丢失的。",
    },
    {
        "input": "我的包不见了，怎么办？",
        "output": "请向工作人员报告，他们将协助您寻找。",
    },
    {
        "input": "我能带我的宠物上飞机吗？",
        "output": "这取决于您的航空公司，请提供您的航空公司名称。",
    },
    {
        "input": "我想知道关于宠物旅行的规定。",
        "output": "请提供您的航空公司名称，我们将为您查询相关的旅行规定。",
    },
]

# 示例选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 示例列表
    examples,
    # 嵌入模型
    OpenAIEmbeddings(),
    # 向量数据库
    FAISS,
    # 要返回的示例数量
    k=2,
    # diversity_score 越高，选择的示例差异越大
    diversity_score_threshold=0.5,
)

# 构建提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"], template="输入: {input}\n输出: {output}\n"
)

# 使用选择器创建 Few-Shot Prompt
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请根据以下示例回答问题。\n\n",
    suffix="输入: {input}\n输出:",
    input_variables=["input"],
)

# 测试示例选择器
print(mmr_prompt.format(input="我的行李箱在哪里？"))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [21]:
from langchain_core.example_selectors import (
    NGramOverlapExampleSelector,
)
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 示例库
examples = [
    {"input": "苹果是一种水果。", "output": "是。"},
    {"input": "香蕉是一种水果。", "output": "是。"},
    {"input": "梨子是一种水果。", "output": "是。"},
    {"input": "汽车是一种水果。", "output": "否。"},
    {"input": "飞机是一种交通工具。", "output": "是。"},
]

# 示例选择器
example_selector = NGramOverlapExampleSelector.from_examples(
    # 示例列表
    examples,
    # 词语重叠度的 n 值
    n=2,
    # k 是要返回的示例数量
    k=2,
)

# 构建示例选择器模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"], template="输入: {input}\n输出: {output}\n"
)

# 使用选择器创建 Few-Shot Prompt
ngram_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请判断以下句子是否正确。\n\n",
    suffix="输入: {input}\n输出:",
    input_variables=["input"],
)

# 测试示例选择器
print(ngram_prompt.format(input="橙子是一种水果。"))

ImportError: cannot import name 'NGramOverlapExampleSelector' from 'langchain_core.example_selectors' (/home/jizhe/projects/python/langchain_learn/.venv/lib/python3.12/site-packages/langchain_core/example_selectors/__init__.py)